# 🚀 Otimização do Modelo Naïve Bayes

Este notebook dá continuidade ao nosso projeto, focando em melhorar a performance do modelo de classificação de spam. Como não podemos adicionar mais dados, vamos otimizar o modelo ajustando o **limiar de probabilidade** para as previsões, buscando reduzir os erros de Falso Positivo.

Vamos seguir os mesmos passos iniciais e, em seguida, adicionar a nossa nova lógica de predição.

In [ ]:
### 1. Importar e Preparar os Dados

# Importar bibliotecas
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Criar o DataFrame com os dados da sua tabela
data = {
    'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'n_palavras_suspeitas': [8, 0, 4, 2, 10, 1, 6, 3, 9, 0],
    'tamanho_email': [500, 120, 300, 250, 800, 150, 400, 200, 600, 100],
    'tem_link': [1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
    'rotulo': [1, 0, 1, 0, 1, 0, 1, 0, 1, 1]
}
df = pd.DataFrame(data)

# Separar características (X) e rótulo (y)
X = df[['n_palavras_suspeitas', 'tamanho_email', 'tem_link']]
y = df['rotulo']

# Dividir os dados em conjuntos de treinamento e teste (mantendo a mesma divisão)
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.3, random_state=42)

# Instanciar e treinar o modelo Naïve Bayes
modelo_bayes = GaussianNB()
modelo_bayes.fit(X_treino, y_treino)

print("Etapas iniciais concluídas: Dados preparados e modelo treinado.")

Etapas iniciais concluídas: Dados preparados e modelo treinado.


### 2. Obter as Probabilidades das Previsões

Em vez de obter a classificação final (`spam` ou `não spam`), vamos obter a **probabilidade** de cada previsão. O método `.predict_proba()` retorna uma matriz com a probabilidade de cada amostra pertencer à classe 0 (não spam) e à classe 1 (spam).

In [ ]:
# Obter as probabilidades para os dados de teste
probabilidades = modelo_bayes.predict_proba(X_teste)

# A primeira coluna é a probabilidade de ser 'Não Spam' (0)
# A segunda coluna é a probabilidade de ser 'Spam' (1)
probabilidades_spam = probabilidades[:, 1]

print("Probabilidades das previsões (classe 'Spam'):")
print(probabilidades_spam)

# Exibir os rótulos reais para comparação
print("\nRótulos Reais do Conjunto de Teste:")
print(y_teste.values)

Probabilidades das previsões (classe 'Spam'):
[1.         0.99994679 0.13158603]

Rótulos Reais do Conjunto de Teste:
[1 0 0]


### 3. Implementar um Limiar de Classificação Personalizado

Com as probabilidades em mãos, podemos definir nossa própria regra de decisão. Para reduzir os Falsos Positivos, vamos exigir uma probabilidade mais alta para classificar um e-mail como spam.

Vamos testar um limiar de **0.8**. Isso significa que a previsão será `SPAM` (1) somente se a probabilidade for maior que `0.8`. Caso contrário, será `NÃO SPAM` (0).

In [ ]:
# Definir o novo limiar
limiar = 0.8

# Criar uma nova lista de previsões baseada no limiar
previsoes_ajustadas = []
for prob in probabilidades_spam:
    if prob > limiar:
        previsoes_ajustadas.append(1)  # Classificar como SPAM
    else:
        previsoes_ajustadas.append(0)  # Classificar como NÃO SPAM

print(f"Previsões ajustadas com limiar de {limiar}:")
print(previsoes_ajustadas)

Previsões ajustadas com limiar de 0.8:
[1, 1, 0]


### 4. Avaliar o Novo Modelo

Agora, vamos calcular as métricas de desempenho novamente, mas usando as **previsões ajustadas** para ver o impacto da nossa mudança.

In [ ]:
# Calcular e imprimir as métricas com as previsões ajustadas
acuracia_ajustada = accuracy_score(y_teste, previsoes_ajustadas)
print(f"\nAcurácia do modelo (ajustada): {acuracia_ajustada:.2f}")

matriz_confusao_ajustada = confusion_matrix(y_teste, previsoes_ajustadas)
print("\nMatriz de Confusão (ajustada):")
print(matriz_confusao_ajustada)

print("\nRelatório de Classificação (ajustado):")
print(classification_report(y_teste, previsoes_ajustadas, target_names=['Não Spam', 'Spam']))


Acurácia do modelo (ajustada): 0.67

Matriz de Confusão (ajustada):
[[1 1]
 [0 1]]

Relatório de Classificação (ajustado):
              precision    recall  f1-score   support

    Não Spam       1.00      0.50      0.67         2
        Spam       0.50      1.00      0.67         1

    accuracy                           0.67         3
   macro avg       0.75      0.75      0.67         3
weighted avg       0.83      0.67      0.67         3

